# Notebook 01 — Dataset Construction from MIMIC-IV / MIMIC-IV-Note

Mục tiêu: xây dựng một tập dữ liệu gọn để huấn luyện mô hình văn bản → đa nhãn (ICD-block và nhóm xét nghiệm giai đoạn sớm), với khả năng giới hạn số dòng đọc ở mọi bước nhằm phục vụ demo nhanh và tiết kiệm tài nguyên.

Quy ước trình bày cho mỗi bước:
1) Mục đích của bước và lý do cần thiết.
2) Trường dữ liệu được sử dụng và ý nghĩa của từng trường; nếu nối giữa các bảng, chỉ rõ khóa nối và ý nghĩa của khóa.
3) Thực hiện xử lý dữ liệu.
4) Diễn giải ý nghĩa các trường trong kết quả hiển thị.

## 0) Cấu hình & nguyên tắc đọc dữ liệu (không dùng `nrows`)

**Quan trọng:** Không tự động giới hạn số dòng bằng `nrows`. Thay vào đó:
- Chọn **COHORT** ngay ở bước setup bằng biến `N_HADM` (số lần nhập viện muốn lấy).
- Ở các bước sau, **đọc theo `chunksize`** và **lọc theo `COHORT_HADM`** để đảm bảo khóa nhất quán giữa admissions/ICD/Lab/Notes.

**Tham số chính trong bước này:**
- `N_HADM` *(int | None)*: số lần nhập viện trong cohort (đặt `None` để dùng toàn bộ).
- `SEED` *(int)*: hạt giống ngẫu nhiên khi cần chọn ngẫu nhiên.
- `BALANCE_BY_SUBJECT` *(bool)*: gợi ý cân bằng theo `subject_id` khi chọn cohort.
- `CHUNKSIZE_DEFAULT` *(int)*: kích thước chunk chuẩn khi đọc các bảng lớn.
- `TOP_LABS` *(int)*: số lab phổ biến (0–6h) dùng làm vocab.

In [7]:
# Cấu hình
from pathlib import Path

# ====== Tham số cohort & đọc file ======
N_HADM = None           # vd: 300_000 hoặc None để dùng toàn bộ admissions
SEED = 42               # reproducibility
BALANCE_BY_SUBJECT = True
CHUNKSIZE_DEFAULT = 500_000  # đọc file lớn theo chunk
TOP_LABS = 50                 # vocab lab phổ biến (0–6h)
LAB_FREQ_WINDOW_HOURS = 240    # cửa sổ giờ để đếm tần suất xét nghiệm
LAB_EARLY_WINDOW_HOURS = 6     # cửa sổ giờ để lấy xét nghiệm sớm nhất
TOP_ICD  = 50                 # vocab ICD-block phổ biến (0–6h)
RAD_EARLY_WINDOW_HOURS = 12    # cửa sổ giờ lấy ghi chú Radiology

# ====== Dò đường dẫn dữ liệu ======
CANDIDATES = [Path("data"), Path("../data"), Path("../../data")]
DATA_ROOT = None
for cand in CANDIDATES:
    if (cand / "mimiciv").exists() and (cand / "mimic-iv-note").exists():
        DATA_ROOT = cand
        break
if DATA_ROOT is None:
    raise FileNotFoundError("Không tìm thấy thư mục 'data' chứa 'mimiciv' và 'mimic-iv-note'. Hãy điều chỉnh CANDIDATES hoặc thiết lập DATA_ROOT thủ công.")

HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
ICU_DIR  = DATA_ROOT / "mimiciv" / "3.1" / "icu"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
PROC_DIR = DATA_ROOT / "proc"; PROC_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_ROOT:", DATA_ROOT.resolve())
print("HOSP_DIR:", HOSP_DIR.resolve())
print("NOTE_DIR:", NOTE_DIR.resolve())
print("PROC_DIR:", PROC_DIR.resolve())
print({
    "N_HADM": N_HADM,
    "SEED": SEED,
    "BALANCE_BY_SUBJECT": BALANCE_BY_SUBJECT,
    "CHUNKSIZE_DEFAULT": CHUNKSIZE_DEFAULT,
    "TOP_LABS": TOP_LABS,
})

DATA_ROOT: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data
HOSP_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/hosp
NOTE_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimic-iv-note/2.2/note
PROC_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc
{'N_HADM': None, 'SEED': 42, 'BALANCE_BY_SUBJECT': True, 'CHUNKSIZE_DEFAULT': 500000, 'TOP_LABS': 50}


### Tiện ích đọc dữ liệu & xử lý cơ bản
- `read_csv_chunks(path, usecols=None, chunksize=CHUNKSIZE_DEFAULT)`: **generator** đọc theo chunk, KHÔNG dùng `nrows`.
- `normalize_text(s)`: chuẩn hóa khoảng trắng, cắt 4,000 ký tự.
- `icd_to_block(icd_code)`: rút gọn mã ICD về block 3 ký tự (bỏ dấu chấm).

**Kết quả mong đợi:**
- In xác nhận đã nạp tiện ích.

**Ý nghĩa trường trả về:**
- `read_csv_chunks(...) → Iterator[DataFrame]`: dùng trong vòng lặp để lọc theo `COHORT_HADM`.
- `normalize_text(s) → str`: văn bản sạch, ổn định tokenize.
- `icd_to_block(code) → str|nan`: nhãn ICD rút gọn, giảm chiều không gian nhãn.

In [8]:
import pandas as pd
import numpy as np
from typing import Iterator, Optional, List

def read_csv_chunks(path, usecols: Optional[List[str]] = None, chunksize: int = CHUNKSIZE_DEFAULT) -> Iterator[pd.DataFrame]:
    """
    Đọc file .csv.gz theo chunk để tiết kiệm RAM. KHÔNG dùng nrows.
    Ví dụ dùng:
        for chunk in read_csv_chunks(labs_path, usecols=["hadm_id","itemid","charttime"]):
            chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
            ...
    """
    return pd.read_csv(
        path,
        usecols=usecols,
        chunksize=chunksize,
        compression="gzip"
    )

def normalize_text(s):
    if pd.isna(s):
        return ""
    return " ".join(str(s).split())[:4000]

def icd_to_block(icd_code: str) -> str:
    if pd.isna(icd_code):
        return np.nan
    s = str(icd_code).replace('.', '').strip()
    return s[:3] if len(s) >= 3 else s

pd.set_option("display.max_colwidth", 180)
pd.set_option("display.width", 140)
print("Đã nạp tiện ích: read_csv_chunks, normalize_text, icd_to_block")

Đã nạp tiện ích: read_csv_chunks, normalize_text, icd_to_block


## 1) Admissions — mốc thời gian nhập viện & khởi tạo COHORT

**Mục đích**
1) Lấy mốc `admittime` cho từng `hadm_id` để tính các cửa sổ sớm: xét nghiệm **0–6h**, ghi chú **0–12h**.
2) **Tạo COHORT_HADM** theo cấu hình ở bước setup (`N_HADM`, `BALANCE_BY_SUBJECT`, `SEED`). Các bảng sau (ICD/Lab/Note) sẽ **lọc theo cohort** khi đọc theo `chunksize`.

**Tham số ảnh hưởng** (đã khai báo ở bước setup):
- `N_HADM` *(int | None)*: số lần nhập viện cần lấy vào cohort. `None` = dùng toàn bộ `admissions`.
- `BALANCE_BY_SUBJECT` *(bool)*: nếu `True`, duyệt theo `subject_id` để hạn chế thiên lệch (nhiều HADM từ cùng một bệnh nhân).
- `SEED` *(int)*: hạt giống ngẫu nhiên khi cần lấy mẫu.

**Kết quả mong đợi**
- `COHORT_HADM` *(set[int])*: tập `hadm_id` cố định dùng xuyên suốt pipeline.
- `admissions_idx` *(Series indexed by `hadm_id`)*: tra cứu nhanh `admittime` phục vụ join thời gian cho Lab/Note.

**Ý nghĩa trường dữ liệu trong kết quả**
- `hadm_id`: mã lần nhập viện — khóa nối giữa các bảng.
- `admittime`: mốc thời gian nhập viện — căn cứ cắt cửa sổ 0–6h (Lab) và 0–12h (Note).
- `subject_id`: mã bệnh nhân (chỉ dùng khi cân bằng cohort theo bệnh nhân).

In [9]:
# --- Bước 1: Đọc admissions & tạo COHORT_HADM ---
import numpy as np
import pandas as pd

adm_path = HOSP_DIR / "admissions.csv.gz"

# Đọc admissions: KHÔNG dùng nrows; chỉ định compression vì file .gz
usecols = ["hadm_id", "admittime"] + (["subject_id"] if BALANCE_BY_SUBJECT or (N_HADM is not None) else [])
admissions = pd.read_csv(adm_path, usecols=usecols, parse_dates=["admittime"], compression="gzip")

# Làm sạch cơ bản
admissions = (
    admissions
    .dropna(subset=["hadm_id", "admittime"])           # cần hadm_id và admittime hợp lệ
    .drop_duplicates(subset=["hadm_id"])                 # mỗi hadm_id duy nhất
)

# Tạo COHORT_HADM theo cấu hình setup
if (N_HADM is None) or (N_HADM >= len(admissions)):
    # Dùng toàn bộ
    COHORT_HADM = set(admissions["hadm_id"].astype(int).tolist())
else:
    rng = np.random.default_rng(SEED)
    if BALANCE_BY_SUBJECT and ("subject_id" in admissions.columns):
        # Duyệt theo subject để phân bổ đều hơn
        subjects = admissions["subject_id"].dropna().astype(int).unique()
        rng.shuffle(subjects)
        picked = []
        for sid in subjects:
            rows = admissions.loc[admissions["subject_id"] == sid, "hadm_id"].astype(int).tolist()
            picked.extend(rows)
            if len(picked) >= N_HADM:
                break
        COHORT_HADM = set(picked[:N_HADM])
    else:
        # Lấy mẫu ngẫu nhiên trực tiếp theo hadm_id
        hadms = admissions["hadm_id"].astype(int).values
        sel = rng.choice(hadms, size=N_HADM, replace=False)
        COHORT_HADM = set(int(x) for x in sel)

# Lọc admissions theo cohort và tạo chỉ mục thời gian
admissions_cohort = admissions[admissions["hadm_id"].astype(int).isin(COHORT_HADM)].copy()
admissions_idx = admissions_cohort.set_index("hadm_id")["admittime"].sort_index()

print("Tổng admissions ban đầu:", len(admissions))
print("Kích thước COHORT_HADM:", len(COHORT_HADM))
display(admissions_idx.to_frame().head())

print("Giải thích kết quả:")
print("- hadm_id (index): mã lần nhập viện trong cohort.")
print("- admittime: mốc thời gian nhập viện để cắt 0–6h (Lab), 0–12h (Note).")

# Gợi ý tiếp theo:
# - Khi đọc ICD/Lab/Notes theo chunksize, luôn lọc: chunk = chunk[chunk['hadm_id'].isin(COHORT_HADM)]
# - Khi cần tham chiếu thời gian: join với admissions_idx theo hadm_id.


Tổng admissions ban đầu: 546028
Kích thước COHORT_HADM: 546028


,admittime
hadm_id,
20000019,2159-03-20 21:08:00
20000024,2151-05-25 21:44:00
20000034,2174-05-22 19:13:00
20000041,2143-09-03 07:15:00
20000045,2138-05-21 16:25:00


Giải thích kết quả:
- hadm_id (index): mã lần nhập viện trong cohort.
- admittime: mốc thời gian nhập viện để cắt 0–6h (Lab), 0–12h (Note).


## 2) ICD-block cho từng lần nhập viện (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**  
- Rút gọn các mã ICD chi tiết thành **ICD-block** (3 ký tự đầu) để giảm số nhãn, vẫn giữ thông tin nhóm bệnh chính.

**Dữ liệu đầu vào & ràng buộc**  
- Dựa trên **COHORT_HADM** đã chọn ở Bước 1.
- Đọc `diagnoses_icd.csv.gz` theo **chunksize** và **lọc theo `hadm_id ∈ COHORT_HADM`** để đảm bảo nhất quán khóa.

**Trường dùng**  
- `subject_id`: mã bệnh nhân  
- `hadm_id`: mã lần nhập viện (khóa nối chính)  
- `icd_code`: mã ICD gốc → chuyển về **ICD-block**

**Kết quả mong đợi**  
- Bảng `icd_df` gồm: `subject_id`, `hadm_id`, `icd_blocks` (danh sách ICD-block **duy nhất** của ca đó).

**Ý nghĩa trường trong kết quả**  
- `icd_blocks`: danh sách các ICD-block (3 ký tự) đại diện nhóm bệnh chính cho mỗi lần nhập viện.

**Lưu ý**  
- Không còn dùng `nrows`; thay vào đó **đọc full theo `chunksize`** và lọc theo **COHORT** ngay trong vòng lặp.

In [10]:
# --- Bước 2: Đọc ICD theo chunksize & lọc theo COHORT_HADM, tạo icd_df ---
import pandas as pd

diag_path = HOSP_DIR / "diagnoses_icd.csv.gz"

icd_rows = []
for chunk in read_csv_chunks(
    diag_path,
    usecols=["subject_id", "hadm_id", "icd_code"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    # làm sạch cơ bản và lọc theo cohort
    chunk = chunk.dropna(subset=["hadm_id", "icd_code"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue
    # chuyển icd_code -> block
    chunk["block"] = chunk["icd_code"].map(icd_to_block)
    icd_rows.append(chunk[["subject_id", "hadm_id", "block"]])

if icd_rows:
    diag_df = pd.concat(icd_rows, ignore_index=True)
else:
    diag_df = pd.DataFrame(columns=["subject_id", "hadm_id", "block"])  # rỗng an toàn

icd_df = (
    diag_df
    .groupby(["subject_id", "hadm_id"]) ["block"]
    .apply(lambda x: sorted({b for b in x if pd.notna(b)}))
    .reset_index()
    .rename(columns={"block": "icd_blocks"})
)

# sanity checks
hadm_icd_n = icd_df["hadm_id"].nunique()
print("ICD — số hadm có nhãn:", hadm_icd_n, "/", len(COHORT_HADM))
display(icd_df.head())

print("Giải thích kết quả:")
print("- subject_id: mã bệnh nhân.")
print("- hadm_id: mã lần nhập viện (khóa nối chính với Lab/Note).")
print("- icd_blocks: danh sách ICD-block (3 ký tự) — nhóm bệnh chính.")

ICD — số hadm có nhãn: 545497 / 546028


,subject_id,hadm_id,icd_blocks
0,10000032,22595853,"[070, 296, 309, 496, 571, 572, 789, V15]"
1,10000032,22841357,"[070, 276, 287, 305, 496, 571, 789, V08]"
2,10000032,25742920,"[070, 276, 305, 496, 571, 787, 789, V08, V46]"
3,10000032,29079034,"[070, 276, 296, 305, 458, 496, 571, 789, 799, V08, V46, V49]"
4,10000068,25022803,[305]


Giải thích kết quả:
- subject_id: mã bệnh nhân.
- hadm_id: mã lần nhập viện (khóa nối chính với Lab/Note).
- icd_blocks: danh sách ICD-block (3 ký tự) — nhóm bệnh chính.


### 2.5) Demographics: giới tính & tuổi tại thời điểm nhập viện

**Mục đích**  
- Tính **tuổi tại nhập viện** và lấy **giới tính** để dùng làm đặc trưng tabular (Bước train).

**Công thức**  
- `age_at_admit = anchor_age + (admit_year - anchor_year)`  
- Sau đó **clip** về `[0, 120]`, **round** và lưu kiểu số nguyên `Int64` (nullable).

**Dữ liệu & ràng buộc**  
- Lấy `subject_id` ↔ `hadm_id` từ `icd_df` (đã lọc theo cohort).
- Năm nhập viện `admit_year` lấy từ `admissions_idx` (Bước 1).  
- Đọc `patients.csv.gz` đầy đủ (file vừa phải) để tính tuổi & lấy giới tính.

**Kết quả mong đợi**  
- Bảng `demo_df` gồm: `hadm_id`, `subject_id`, `gender` (M/F/U), `age_at_admit` (0–120, Int64).

In [11]:
# --- Bước 2.5: Tính demographics (gender, age_at_admit) ---
pat_path = HOSP_DIR / "patients.csv.gz"

# Đọc demographics
patients = pd.read_csv(
    pat_path,
    usecols=["subject_id", "gender", "anchor_age", "anchor_year"],
    compression="gzip",
)
patients["gender"] = patients["gender"].astype(str).str.upper().str[0]  # 'M'/'F'/'U'

# Map hadm_id -> subject_id từ icd_df
hadm_subject = icd_df[["hadm_id", "subject_id"]].drop_duplicates()

# Lấy năm nhập viện từ admissions_idx (Series indexed by hadm_id)
adm_year = admissions_idx.to_frame(name="admittime").reset_index()
adm_year["admit_year"] = adm_year["admittime"].dt.year

# Ghép để tính tuổi tại nhập viện
age_df = (
    hadm_subject
    .merge(patients, on="subject_id", how="left")
    .merge(adm_year[["hadm_id", "admit_year"]], on="hadm_id", how="left")
)
age_df["age_at_admit"] = age_df["anchor_age"] + (age_df["admit_year"] - age_df["anchor_year"])
age_df["age_at_admit"] = (
    age_df["age_at_admit"].clip(lower=0, upper=120).round().astype("Int64")
)

demo_df = age_df[["hadm_id", "subject_id", "gender", "age_at_admit"]].copy()

# sanity checks
print("Demo — số hadm có demographics:", demo_df["hadm_id"].nunique())
display(demo_df.head())

print("Giải thích kết quả:")
print("- gender: M/F/U.")
print("- age_at_admit: tuổi tại thời điểm nhập viện (0–120, kiểu Int64).")

Demo — số hadm có demographics: 545497


,hadm_id,subject_id,gender,age_at_admit
0,22595853,10000032,F,52
1,22841357,10000032,F,52
2,25742920,10000032,F,52
3,29079034,10000032,F,52
4,25022803,10000068,F,19


Giải thích kết quả:
- gender: M/F/U.
- age_at_admit: tuổi tại thời điểm nhập viện (0–120, kiểu Int64).


## 3) Nhóm xét nghiệm trong 6 giờ đầu (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**
- Xây dựng nhãn về **các xét nghiệm được thực hiện sớm** (0–6 giờ đầu kể từ `admittime`) để huấn luyện mô hình gợi ý cận lâm sàng.

**Dữ liệu đầu vào & ràng buộc**
- `labevents.csv.gz`: `hadm_id`, `itemid`, `charttime`  
- `d_labitems.csv.gz`: `itemid` → `label`  
- `admissions_idx` (Bước 1): tra cứu `admittime` theo `hadm_id`  
- **Chỉ lấy bản ghi có `hadm_id ∈ COHORT_HADM`**. Đọc theo `chunksize` để tiết kiệm RAM.

**Xử lý (2 pass)**
1) **Đếm tần suất** `itemid` trong 0–6h để chọn `TOP_LABS` (vocab xét nghiệm sớm phổ biến).  
2) **Gán nhãn cho từng ca**: với mỗi `hadm_id`, lấy tập hợp `itemid` (thuộc vocab) diễn ra trong 0–6h.

**Kết quả mong đợi**
- `lab_vocab_df`: bảng vocab xét nghiệm sớm gồm `itemid`, `label`, `count` (tần suất).  
- `lab_items_by_hadm`: cho *mỗi* `hadm_id`, danh sách duy nhất các `itemid` (thuộc vocab) xuất hiện trong 0–6h.  
- (Tiện ích) `itemid_to_label`: ánh xạ `itemid → label` để hiển thị dễ đọc.

**Ý nghĩa trường trong kết quả**
- `itemid`: mã xét nghiệm.  
- `label`: tên xét nghiệm.  
- `count`: số lần xuất hiện trong cửa sổ 0–6h (trên toàn cohort).  
- `lab_items` (ở `lab_items_by_hadm`): danh sách *duy nhất* các `itemid` thuộc vocab của từng ca.

In [12]:
# --- Bước 3: Lấy xét nghiệm sớm Top-N theo tần suất ---
from collections import Counter
import pandas as pd

labs_path = HOSP_DIR / "labevents.csv.gz"
dlab_path = HOSP_DIR / "d_labitems.csv.gz"

# 3.1 Đọc dictionary lab: itemid -> label
dlab = pd.read_csv(dlab_path, usecols=["itemid", "label"], compression="gzip")
dlab["itemid"] = dlab["itemid"].astype(int)
dlab["label"] = dlab["label"].astype(str)

# Chuẩn bị bảng tra admittime theo hadm_id (từ Bước 1)
adm_df = admissions_idx.to_frame(name="admittime").reset_index()

# ===== Pass 1: Đếm tần suất itemid trong cohort (0–LAB_FREQ_WINDOW_HOURS) =====
cnt = Counter()

for chunk in read_csv_chunks(
    labs_path,
    usecols=["hadm_id", "itemid", "charttime"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    chunk = chunk.dropna(subset=["hadm_id", "itemid", "charttime"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk["itemid"]  = chunk["itemid"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue
    chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")

    merged = chunk.merge(adm_df, on="hadm_id", how="inner")
    dt_h = (merged["charttime"] - merged["admittime"]).dt.total_seconds() / 3600.0
    early = merged[(dt_h >= 0) & (dt_h <= LAB_FREQ_WINDOW_HOURS)]
    if early.empty:
        continue
    cnt.update(early["itemid"].value_counts().to_dict())

counts_df = (
    pd.DataFrame(list(cnt.items()), columns=["itemid", "count"]) if cnt else
    pd.DataFrame(columns=["itemid", "count"])
)
counts_df = counts_df.sort_values(["count", "itemid"], ascending=[False, True])
top_items = counts_df.head(TOP_LABS)["itemid"].astype(int).tolist()

lab_vocab_df = (
    counts_df[counts_df["itemid"].isin(top_items)]
    .merge(dlab, on="itemid", how="left")
    [["itemid", "label", "count"]]
    .reset_index(drop=True)
)

# ===== Pass 2: Gán nhãn theo hadm_id (0–LAB_EARLY_WINDOW_HOURS, giữ itemid thuộc vocab) =====
lab_rows = []

if len(top_items) > 0:
    for chunk in read_csv_chunks(
        labs_path,
        usecols=["hadm_id", "itemid", "charttime"],
        chunksize=CHUNKSIZE_DEFAULT,
    ):
        chunk = chunk.dropna(subset=["hadm_id", "itemid", "charttime"]).copy()
        if chunk.empty:
            continue
        chunk["hadm_id"] = chunk["hadm_id"].astype(int)
        chunk["itemid"]  = chunk["itemid"].astype(int)
        chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
        if chunk.empty:
            continue
        chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")

        merged = chunk.merge(adm_df, on="hadm_id", how="inner")
        dt_h = (merged["charttime"] - merged["admittime"]).dt.total_seconds() / 3600.0
        early = merged[(dt_h >= 0) & (dt_h <= LAB_EARLY_WINDOW_HOURS)]
        early_top = early[early["itemid"].isin(top_items)]
        if not early_top.empty:
            lab_rows.append(early_top[["hadm_id", "itemid"]])

if lab_rows:
    labs_concat = pd.concat(lab_rows, ignore_index=True)
    lab_items_by_hadm = (
        labs_concat
        .groupby("hadm_id")["itemid"]
        .apply(lambda s: sorted(set(s.tolist())))
        .reset_index()
        .rename(columns={"itemid": "lab_items"})
    )
else:
    lab_items_by_hadm = pd.DataFrame(columns=["hadm_id", "lab_items"])  # rỗng an toàn

# Ánh xạ itemid -> label để hiển thị thuận tiện ở các bước sau
itemid_to_label = dict(zip(lab_vocab_df["itemid"], lab_vocab_df["label"]))

# Báo cáo nhanh
print("Số phần tử vocab xét nghiệm (TOP_N):", len(lab_vocab_df))
print("Số ca có ít nhất một xét nghiệm trong TOP_N:", lab_items_by_hadm["hadm_id"].nunique())
display(lab_vocab_df.head(10))
display(lab_items_by_hadm.head(10))

# ===== 3.x Lưu danh sách Top Labs ra CSV =====
out_top_labs = PROC_DIR / "top_lab_items.csv"
lab_vocab_df.to_csv(out_top_labs, index=False)
print("Đã lưu danh sách Top Labs:", out_top_labs)


Số phần tử vocab xét nghiệm (TOP_N): 50
Số ca có ít nhất một xét nghiệm trong TOP_N: 172729


,itemid,label,count
0,50971,Potassium,2031751
1,50983,Sodium,2012609
2,51221,Hematocrit,2004337
3,50902,Chloride,1994684
4,50912,Creatinine,1985772
5,51006,Urea Nitrogen,1968678
6,50882,Bicarbonate,1949085
7,50868,Anion Gap,1944213
8,50931,Glucose,1923325
9,51265,Platelet Count,1895054


,hadm_id,lab_items
0,20000147,"[50802, 50804, 50808, 50809, 50813, 50818, 50820, 50821, 50822, 50868, 50882, 50902, 50912, 50934, 50947, 50960, 51006, 51146, 51200, 51221, 51222, 51237, 51244, 51248, 51249, ..."
1,20000159,"[50868, 50882, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301]"
2,20000261,"[51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301]"
3,20000293,"[50868, 50882, 50893, 50902, 50912, 50920, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51237, 51244, 51248, 51249, 51250, 51254, 51256, 51265, 51274, ..."
4,20000298,[51003]
5,20000343,"[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51003, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, ..."
6,20000394,"[50971, 51265]"
7,20000629,"[50861, 50862, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50931, 50954, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51237, 51244, 51248, 51249, 51250, ..."
8,20000768,"[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301, 51678, 52172]"
9,20000769,"[50861, 50878, 50912, 50920, 50934, 50947, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301, 51678, 52172]"


Đã lưu danh sách Top Labs: ../data/proc/top_lab_items.csv


## 4) Ghi chú Radiology trong 12 giờ đầu (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**  
- Lấy **ghi chú radiology sớm nhất** trong **0–12 giờ** kể từ `admittime` cho mỗi `hadm_id` trong **COHORT_HADM**, làm đầu vào mô hình text→label.

**Dữ liệu đầu vào & ràng buộc**  
- `radiology.csv.gz`: `hadm_id`, `charttime`, `text`  
- `admissions_idx` (Bước 1): tra cứu `admittime` theo `hadm_id`  
- **Chỉ lấy bản ghi có `hadm_id ∈ COHORT_HADM`**. Đọc theo `chunksize` để tiết kiệm RAM.

**Xử lý**  
1) Đọc radiology theo `chunksize`, lọc theo `COHORT_HADM`.  
2) Join `admittime` từ `admissions_idx`, tính \(\Delta t\) (giờ).  
3) Giữ các ghi chú có \(0 \le \Delta t \le 12\).  
4) `normalize_text` nội dung và **chọn ghi chú sớm nhất** (nhỏ nhất `charttime`) cho mỗi `hadm_id`.

**Kết quả mong đợi**  
- `text_df`: DataFrame gồm `hadm_id`, `text` (radiology note sớm nhất trong 12h, đã chuẩn hoá).

**Ý nghĩa trường trong kết quả**  
- `hadm_id`: mã lần nhập viện (khóa chính để join).  
- `text`: văn bản ghi chú radiology đầu tiên trong 12h đầu, đã làm sạch để sẵn sàng tokenize.

In [13]:
# --- Bước 4: Radiology, lấy ghi chú sớm nhất mỗi HADM ---
import pandas as pd

rad_path = NOTE_DIR / "radiology.csv.gz"

# Chuẩn bị bảng tra admittime (từ Bước 1)
adm_df = admissions_idx.to_frame(name="admittime").reset_index()

early_rows = []
for chunk in read_csv_chunks(
    rad_path,
    usecols=["hadm_id", "charttime", "text"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    # Lọc & làm sạch cơ bản
    chunk = chunk.dropna(subset=["hadm_id", "charttime", "text"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue
    chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")
    chunk["text"] = chunk["text"].map(normalize_text)

    # Join admittime và tính delta giờ
    merged = chunk.merge(adm_df, on="hadm_id", how="inner")
    dt_h = (merged["charttime"] - merged["admittime"]).dt.total_seconds() / 3600.0
    early = merged[(dt_h >= 0) & (dt_h <= RAD_EARLY_WINDOW_HOURS)]
    if not early.empty:
        early_rows.append(early[["hadm_id", "charttime", "text"]])

if early_rows:
    rad_early = pd.concat(early_rows, ignore_index=True)
    # Chọn ghi chú sớm nhất theo charttime cho mỗi hadm_id
    rad_early = rad_early.sort_values(["hadm_id", "charttime"], ascending=[True, True])
    first_text = rad_early.groupby("hadm_id", as_index=False)["text"].first()
    text_df = first_text[["hadm_id", "text"]].copy()
else:
    text_df = pd.DataFrame(columns=["hadm_id", "text"])  # rỗng an toàn

print(f"Radiology — số hadm có note 0–{RAD_EARLY_WINDOW_HOURS}h:", text_df["hadm_id"].nunique())
display(text_df.head())
print("Giải thích kết quả:")
print("- hadm_id: mã lần nhập viện (trùng khóa chuẩn).")
print(f"- text: ghi chú radiology sớm nhất trong {RAD_EARLY_WINDOW_HOURS} giờ đầu, đã normalize.")


Radiology — số hadm có note 0–12h: 115696


,hadm_id,text
0,20000057,"CLINICAL HISTORY: Fell on buttocks, chronic left hip osteoarthritis. Evaluate for fracture. PELVIS AND LEFT HIP Total hip replacement is present on the right side. This appears..."
1,20000147,"INDICATION: ___ year old man s/p Emergent CABG. Please ___ at ___ with abnormalities. // FAST TRACK EXTUBATION CARDIAC SURGERY, ?line placement, r/o PTX/Effusion Contact name: ..."
2,20000235,"INDICATION: ___ man with right and left lower extremity swelling. COMPARISON: No previous exam for comparison. FINDINGS: Grayscale, color and Doppler images were obtained of bi..."
3,20000254,INDICATION: Evaluate for obstruction in a ___ woman with abdominal pain and constipation. TECHNIQUE: Frontal supine and upright abdominal radiographs were obtained. COMPARISON:...
4,20000293,REASON FOR EXAMINATION: Preoperative evaluation in a patient with right foot ulceration. PA and lateral upright chest radiograph was reviewed with no prior studies available fo...


Giải thích kết quả:
- hadm_id: mã lần nhập viện (trùng khóa chuẩn).
- text: ghi chú radiology sớm nhất trong 12 giờ đầu, đã normalize.


## 5) Ghép dữ liệu & sinh nhãn đa nhãn (multi-hot) — **theo COHORT**

**Mục đích**  
- Gom các phần: **ICD-block** (Bước 2), **lab sớm 0–6h** (Bước 3), **radiology 0–12h** (Bước 4), **demographics** (Bước 2.5) thành **một bảng mẫu/nhãn** dùng cho huấn luyện.

**Khóa nối & đầu vào**  
- Nối theo `hadm_id` giữa `icd_df`, `lab_items_by_hadm`, `text_df`.  
- Thêm demographics từ `demo_df` theo cặp khóa `[hadm_id, subject_id]`.

**Xử lý**  
1) Chuẩn hóa các cột list/text/demographics.  
2) Tạo **vocabulary**:  
   • `icd_vocab`: danh sách ICD-block phổ biến (giới hạn bởi `ICD_MAX`).  
   • `lab_vocab_items`: chính là Top-N ở Bước 3 (giữ nguyên thứ tự).  
3) Sinh vector **multi-hot** cho mỗi hàng: `y_icd`, `y_lab`.

**Kết quả mong đợi**  
- DataFrame `df_out` gồm: `hadm_id`, `subject_id`, `gender`, `age_at_admit`, `text`, `icd_blocks`, `lab_items`, `y_icd`, `y_lab`.  
- Ghi file:  
  • `examples.parquet`  
  • `vocab_meta.json` (lưu vocab & mapping `itemid→label`).

**Ý nghĩa các trường chính**  
- `icd_blocks`: danh sách ICD-block (3 ký tự).  
- `lab_items`: danh sách *duy nhất* các `itemid` thuộc vocab xuất hiện trong 0–6h.  
- `y_icd`/`y_lab`: vector multi-hot tương ứng với `icd_vocab`/`lab_vocab_items`.  
- `gender` (`M/F/U`), `age_at_admit` (0–120, `Int64`).

In [14]:
# --- Bước 5: Merge & sinh nhãn multi-hot, rồi lưu ra PROC_DIR ---
import numpy as np
import pandas as pd
from collections import Counter
import json

# 5.1 Ghép theo hadm_id (và subject_id cho demographics)
df = (
    icd_df
    .merge(lab_items_by_hadm, on="hadm_id", how="left")
    .merge(text_df,            on="hadm_id", how="left")
    .merge(demo_df,            on=["hadm_id", "subject_id"], how="left")
)

# 5.2 Chuẩn hoá các cột
df["icd_blocks"] = df["icd_blocks"].apply(lambda x: x if isinstance(x, list) else [])
df["lab_items"]  = df["lab_items"].apply(lambda x: x if isinstance(x, list) else [])
df["text"]       = df["text"].fillna("")
df["gender"]     = df["gender"].fillna("U").astype(str).str.upper().str[0]  # M/F/U
df["age_at_admit"] = df["age_at_admit"].astype("Int64")

# Chỉ giữ lần nhập viện có note
df["text"] = df["text"].str.strip()
df = df[df["text"].str.len() > 0].copy()

# 5.3 Vocabulary
cnt_icd = Counter(b for blocks in df["icd_blocks"] for b in blocks)
icd_vocab = [b for b, _ in cnt_icd.most_common(TOP_ICD)]
icd_index = {b: i for i, b in enumerate(icd_vocab)}

# lab vocab kế thừa từ bước 3 (giữ thứ tự theo tần suất)
lab_vocab_items = lab_vocab_df["itemid"].astype(int).tolist()
lab_index = {it: i for i, it in enumerate(lab_vocab_items)}

# 5.4 Hàm multi-hot
def to_multihot_generic(labels, index_map, length):
    arr = np.zeros(length, dtype=np.int8)
    for t in labels:
        if t in index_map:
            arr[index_map[t]] = 1
    return arr

df["y_icd"] = df["icd_blocks"].apply(lambda xs: to_multihot_generic(xs, icd_index, len(icd_vocab)))
df["y_lab"] = df["lab_items"].apply(lambda xs: to_multihot_generic(xs, lab_index, len(lab_vocab_items)))

# 5.5 Ánh xạ itemid -> label để tiện hiển thị/giải thích
itemid_to_label = dict(zip(lab_vocab_df["itemid"].astype(int), lab_vocab_df["label"].astype(str)))

print("Kích thước vocab ICD:", len(icd_vocab))
print("Kích thước vocab Lab (Top-N):", len(lab_vocab_items))
display(df.head())

# 5.6 Lưu ra đĩa (sau khi 5.2 đã lọc chỉ còn các ca có note)
PROC_DIR.mkdir(parents=True, exist_ok=True)

out_examples = PROC_DIR / "examples.parquet"
df_out = df[[
    "hadm_id", "subject_id", "gender", "age_at_admit",
    "text", "icd_blocks", "lab_items", "y_icd", "y_lab"
]].copy()
df_out.to_parquet(out_examples, index=False)

out_vocab = PROC_DIR / "vocab_meta.json"
with open(out_vocab, "w") as f:
    json.dump({
        "icd_vocab": icd_vocab,
        "n_icd": len(icd_vocab),
        "lab_vocab_items": lab_vocab_items,
        "n_lab": len(lab_vocab_items),
        "itemid_to_label": {int(k): str(v) for k, v in itemid_to_label.items()}
    }, f, indent=2)

print("Đã lưu (chỉ các HADM có note):")
print("-", out_examples)
print("-", out_vocab)

# 5.7 Xuất Top ICD block chi tiết kèm số ca bệnh (n_hadm)
diag_dict_path = HOSP_DIR / "d_icd_diagnoses.csv.gz"
d_icd_diag = pd.read_csv(
    diag_dict_path,
    usecols=["icd_code", "icd_version", "long_title"],
    compression="gzip"
)
d_icd_diag["icd_code"] = d_icd_diag["icd_code"].astype(str)

# Tính số ca bệnh unique cho mỗi block
hadm_per_block = (
    icd_df.explode("icd_blocks")[["hadm_id", "icd_blocks"]]
    .dropna()
    .drop_duplicates()
    .groupby("icd_blocks")["hadm_id"]
    .nunique()
    .to_dict()
)

rows = []
for block, freq in cnt_icd.most_common(TOP_ICD):
    related = d_icd_diag[d_icd_diag["icd_code"].str.startswith(block)]
    n_hadm = hadm_per_block.get(block, 0)
    if related.empty:
        rows.append({
            "block": block,
            "frequency": freq,
            "n_hadm": n_hadm,
            "icd_code": "",
            "long_title": "(not found)"
        })
    else:
        for _, r in related.iterrows():
            rows.append({
                "block": block,
                "frequency": freq,
                "n_hadm": n_hadm,
                "icd_code": r["icd_code"],
                "long_title": r["long_title"]
            })

top_icd_df = pd.DataFrame(rows)
out_icd_blocks = PROC_DIR / "top_icd_blocks.csv"
top_icd_df.to_csv(out_icd_blocks, index=False)

print("Đã lưu danh sách Top ICD block chi tiết:", out_icd_blocks)


Kích thước vocab ICD: 50
Kích thước vocab Lab (Top-N): 50


,subject_id,hadm_id,icd_blocks,lab_items,text,gender,age_at_admit,y_icd,y_lab
0,10000032,22595853,"[070, 296, 309, 496, 571, 572, 789, V15]",[],"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year-old female with cirrhosis, jaundice. TECHNIQUE: Grey scale and color Doppler ultrasound images of the a...",F,52,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000032,25742920,"[070, 276, 305, 496, 571, 787, 789, V08, V46]",[],"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year old woman with cirrhosis, p/w abd pain as well as elevated alk phos // eval for biliary pathology and P...",F,52,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
21,10000764,27897940,"[272, 287, 288, 401, 410, 412, 414, 424, 443, 584, 780, 784, 799, 802, E84, E88, V15, V58]",[51221],"EXAMINATION: CHEST (PA AND LAT) INDICATION: ___ with hypoxia // ?pna, aspiration. COMPARISON: None FINDINGS: PA and lateral views of the chest provided. The lungs are adequatel...",M,86,"[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
36,10000980,20897796,"[D64, E11, E66, E78, E87, G47, I13, I25, I50, M1A, N18, N25, R09, Z22, Z68, Z79, Z86, Z95]","[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51003, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, ...","EXAMINATION: BILAT LOWER EXT VEINS INDICATION: ___ year old woman with hx CAD, HFpEF, PVD, prior b/l DVT here with acute onset SOB, most likely heart failure, want to rule out ...",F,80,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
48,10001217,27703517,"[041, 305, 324, 340, 348, 401]",[],"CHEST RADIOGRAPH INDICATION: PICC line placement. COMPARISON: ___. FINDINGS: As compared to the previous radiograph, there is evidence of an unchanged PICC line placed over the...",F,55,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


Đã lưu (chỉ các HADM có note):
- ../data/proc/examples.parquet
- ../data/proc/vocab_meta.json
Đã lưu danh sách Top ICD block chi tiết: ../data/proc/top_icd_blocks.csv


In [15]:
# Kiểm tra 5 mẫu có lab_items KHÔNG rỗng (và y_lab có ít nhất 1 nhãn)
import numpy as np
import pandas as pd

def has_lab_items(x):
    return isinstance(x, (list, tuple)) and len(x) > 0

def has_y_lab(x):
    try:
        return np.asarray(x).sum() > 0
    except Exception:
        return False

# Nếu bạn đã lưu ra df_out (ở Bước 5), thì đọc lại; còn nếu đang có biến df thì dùng df
try:
    _ = df
    frame = df
except NameError:
    frame = pd.read_parquet(PROC_DIR / "examples.parquet")

mask_items = frame["lab_items"].apply(has_lab_items)
mask_y     = frame["y_lab"].apply(has_y_lab)

nonempty_lab = frame[mask_items].copy()
print(f"Số ca có lab_items không rỗng: {len(nonempty_lab)} / {len(frame)} "
      f"({len(nonempty_lab)/len(frame):.1%})")

nonempty_both = frame[mask_items & mask_y].copy()
print(f"Số ca có lab_items≠∅ và y_lab>0: {len(nonempty_both)}")

# Lấy ngẫu nhiên 5 mẫu để xem chi tiết
sample_n = min(5, len(nonempty_lab))
sample_rows = nonempty_lab.sample(sample_n, random_state=42) if sample_n > 0 else nonempty_lab.head(0)

cols_show = ["subject_id", "hadm_id", "gender", "age_at_admit", "icd_blocks", "lab_items", "text"]
display(sample_rows[cols_show])

# (Tuỳ chọn) In kèm tổng số nhãn lab được bật để xác nhận khớp y_lab
if sample_n > 0:
    tmp = sample_rows.copy()
    tmp["y_lab_sum"] = tmp["y_lab"].apply(lambda a: int(np.asarray(a).sum()) if isinstance(a, (list,np.ndarray)) else 0)
    display(tmp[["hadm_id", "lab_items", "y_lab_sum"]])


Số ca có lab_items không rỗng: 55726 / 115657 (48.2%)
Số ca có lab_items≠∅ và y_lab>0: 55726


,subject_id,hadm_id,gender,age_at_admit,icd_blocks,lab_items,text
374973,16882065,25006263,F,69,"[D62, E11, E87, I10, J98, N39, S00, S06, S30, S32, V03, Y92, Z78, Z86]","[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301, ...","EXAMINATION: CTA HEAD AND CTA NECK INDICATION: ___ woman pedestrian struck by motor vehicle with known SAH, IPH. Per neurosurgery, pattern of bleed suspicious for cerebral aneu..."
49806,10937007,21698987,F,83,"[B96, D64, E11, E87, F03, G81, G92, I10, I60, I97, J18, K56, N39, R09, R33, R47, R56, S06, W19, Y83, Y92, Z78, Z91]","[50802, 50804, 50808, 50813, 50818, 50820, 50821, 50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, ...","EXAMINATION: CHEST (PORTABLE AP) INDICATION: ___ year old woman with SDH, intubated// eval ETT and OGT position eval ETT and OGT position IMPRESSION: Compared to chest radiogra..."
201351,13707062,20101799,M,77,"[250, 272, 276, 285, 404, 412, 414, 427, 428, 458, 496, 537, 578, 584, 585, 600, 780, 790, V45, V58]","[51221, 51222]",CHEST RADIOGRAPH PERFORMED ON ___ COMPARISON: Prior exam from ___. CLINICAL HISTORY: Shortness of breath. FINDINGS: PA and lateral views of the chest provided demonstrating a l...
119216,12212328,24934171,M,57,"[C77, C78, C79, D72, G89, K21, R11, Z80, Z85, Z87, Z90]","[50861, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50920, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51237, 51244, 51248, ...",EXAMINATION: CHEST (PORTABLE AP) INDICATION: ___ year old man with metastatic bladder cancer and somnolence // Evidence of infection? Evidence of infection? IMPRESSION: Heart s...
70555,11317338,23243071,F,51,"[278, 311, 493, 564, 574, 715, 783, V45, V85]","[50861, 50862, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, ...","EXAMINATION: CT ABDOMEN AND PELVIS INDICATION: History of Roux-en-Y bypass, status post lap cholecystectomy and clipping of the duct of Luschka, POD 3. Question fluid collectio..."


,hadm_id,lab_items,y_lab_sum
374973,25006263,"[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301, ...",27
49806,21698987,"[50802, 50804, 50808, 50813, 50818, 50820, 50821, 50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, ...",35
201351,20101799,"[51221, 51222]",2
119216,24934171,"[50861, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50920, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51237, 51244, 51248, ...",37
70555,23243071,"[50861, 50862, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, ...",30
